In [2]:
import openai, os, chromadb, json

openai.api_key = os.environ.get('OPENAI_API_KEY')

In [4]:
chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name="my_collection")

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [5]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":1000,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()

In [6]:
theme = "dystopian universe"

In [7]:
generatePlotPrompt = """ /* Generate an Plot for a movie

type character{{
  "name": str,
  "background": str
}}

interface overview {{
  "setting": str, //where the section takes place
  "characters": character[], //the character names
  "conflict": str, //what conflict needs to get solved
  "plotTwist": str, //any plot twists to occur
  "resolution": str // how this plot concludes
}}

const generatePlot = (theme) : outline => {{
    return ai.generatePlot(theme)
}}


theme = '{}'
//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(theme))
Out:"""

In [8]:
generateScenesPrompt = """ /* Generate scenes for a movie

type scene{{
  "name": str, // the name of the scene
  "summary": str, // a detailed description with specifics of what happens in this scene
}}

const generateScenes = (plot) : scene[] =>{{
    return ai.generateScenes(plot)
}}

plot = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(plot))
Out:"""

In [9]:
generateDialoguesPrompt = """ /* Write the next set of dialogues in the given scene of the movie

//the overview of the entire movie
movieOverview = {}

//the outline that the movie should follow
movieOutline = {}

//the paragraphs of the movie so far
dialoguesSoFar = {}

interface scene {{
  "dialogues": dialogue[] //all of the dialogues in the story
}}

type dialogue{{
  "name": str, // the name of the speaker
  "text": str, // dialogue that speaker says
}}

//generate multiple longform dialogues for this scene and list them
const {} = (currentScene) : dialogues[] =>{{
    return ai.write()
}}

currentSceneDescription = {}

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + {}(currentOutlineSection))
Out:"""

In [10]:
generateDialoguesPrompt = """ /* Write the next set of dialogues in the movie for the given scene

//the Plot of the entire movie
plot = {}

//the outline that the movie should follow
allScenes = {}

//list of dialogues that have already been spoken
dialoguesSoFar = {}

type dialogue{{
  "name": str, // the name of the speaker
  "text": str, // dialogue that speaker says
}}

//generate multiple longform dialogues for this scene and list them
const {} = (currentScene) : dialogue[] =>{{
    return ai.write()
}}

currentSceneDescription = {}

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + {}(currentOutlineSection))
Out:"""

In [12]:
from uuid import uuid4

In [16]:
rawPlot = query(generatePlotPrompt.format(theme), temperature=0.5)
parsedPlot = json.loads(rawPlot)
print('Overall Plot')
display(parsedPlot)

rawScenes = query(generateScenesPrompt.format(rawPlot), temperature=.5)
parsedScenes = json.loads(rawScenes)
print('Overall Scenes')
display(parsedScenes)

dialoguesSoFar = []

for scenedidx, scene in enumerate(parsedScenes):

    subTheme = f"{scene['name']}: {scene['summary']}\n"
    print(subTheme)

    rawSubPlot = query(generatePlotPrompt.format(subTheme), temperature=0.5)
    parsedSubPlot = json.loads(rawSubPlot)
    print('Sub Plot')
    display(parsedSubPlot)
    
    rawSubScenes = query(generateScenesPrompt.format(rawSubPlot), temperature=.5)
    parsedSubScenes = json.loads(rawSubScenes)
    print('Sub Scenes')
    display(parsedSubScenes)

    for subScene in parsedSubScenes:
        try:
            similarDialogues = collection.query(
                query_texts=[subScene['summary']],
                n_results=2
            )
            
            print('similarDialogues', similarDialogues)

            rawDialogues = query(generateDialoguesPrompt.format(rawSubPlot, rawSubScenes, dialoguesSoFar[-10:], subScene['name'], subScene['summary'], subScene['name']))
            print(f'Dialogues generated: {len(json.loads(rawDialogues))}')
            parsedDialoges = json.loads(rawDialogues)
            recordedDialoges = []
            for dialogue in parsedDialoges:
                dialogue['id'] = str(uuid4().hex)
                collection.add(documents=[json.dumps(dialogue)], metadatas=[{"source": "my_source"}], ids=[dialogue['id']])
                recordedDialoges.append(dialogue)
            
            dialoguesSoFar += recordedDialoges
        except:
            try:
                print('FAILED, TRYING AGAIN')
                rawDialogues = query(generateDialoguesPrompt.format(rawSubPlot, rawSubScenes, dialoguesSoFar[-10:], subScene['name'], subScene['summary'], subScene['name']))
                print(f'Dialogues generated: {len(json.loads(rawDialogues))}')
                recordedDialoges = []
                for dialogue in parsedDialoges:
                    dialogue['id'] = str(uuid4().hex)
                    collection.add(documents=[json.dumps(dialogue)], metadatas=[{"source": "my_source"}], ids=[dialogue['id']])
                    recordedDialoges.append(dialogue)
                
                dialoguesSoFar += recordedDialoges
            except:
                pass

    for dialogue in dialoguesSoFar:
        print(f"{dialogue['name']}: {dialogue['text']}")


Overall Plot


{'setting': 'A dystopian universe in the near future',
 'characters': [{'name': 'John',
   'background': 'A young man struggling to find his place in the world'},
  {'name': 'Samantha', 'background': 'A powerful leader of the resistance'}],
 'conflict': "John must choose between joining Samantha's rebellion or supporting the oppressive regime",
 'plotTwist': 'John discovers that Samantha is his long lost sister',
 'resolution': 'John joins the rebellion, and they overthrow the oppressive regime together'}

Overall Scenes


[{'name': 'Meeting John',
  'summary': 'John meets Samantha for the first time and is immediately taken by her strength and courage. He learns about her mission to overthrow the oppressive regime and is intrigued by her cause.'},
 {'name': "John's Choice",
  'summary': "John is faced with a difficult decision - join Samantha's rebellion or stay loyal to the oppressive regime. He is torn between his loyalty and his desire to join the cause of justice."},
 {'name': 'The Reveal',
  'summary': 'John discovers that Samantha is actually his long lost sister. He is filled with a newfound sense of purpose and decides to join the rebellion.'},
 {'name': 'The Final Battle',
  'summary': 'John and Samantha lead the rebellion against the oppressive regime. After a long and difficult battle, they are successful in overthrowing the regime and restoring peace and justice to the universe.'}]

Meeting John: John meets Samantha for the first time and is immediately taken by her strength and courage. He learns about her mission to overthrow the oppressive regime and is intrigued by her cause.

Sub Plot


...

{'setting': "John's home town",
 'characters': [{'name': 'John', 'background': 'An ordinary person'},
  {'name': 'Samantha', 'background': "John's long lost sister"}],
 'conflict': "John must uncover the truth about his sister's identity and decide whether or not to join the rebellion.",
 'plotTwist': 'John discovers that Samantha is actually his long lost sister and that she is part of the rebellion.',
 'resolution': 'John decides to join the rebellion and fight for justice alongside his sister.'}

Sub Scenes


[{'name': "John's Discovery",
  'summary': "John discovers a clue about Samantha's identity and decides to investigate further. He goes to the library and finds out that Samantha is actually his long lost sister and is part of the rebellion."},
 {'name': "John's Decision",
  'summary': 'John must decide whether to join the rebellion or not. After a long and difficult deliberation, he decides to join and fight for justice alongside his sister.'},
 {'name': 'The Final Battle',
  'summary': 'John and Samantha join forces with the rebellion to fight against the oppressive regime. After a long and difficult battle, they emerge victorious and the oppressive regime is overthrown.'}]

Dialogues generated: 6
Dialogues generated: 12
Dialogues generated: 6
John: This is a terrible place. I can feel the oppression in the air. I must find a way to help these people.
Citizen: You can't do anything. The regime is too powerful. We are all doomed.
John: No, I won't accept that. There must be a way to fight back. I will find it.
Citizen: You are brave, but it won't be enough. We need a miracle.
John: Hello, I'm John. I've heard about the oppressive regime here and I want to help.
Samantha: I'm Samantha. I'm leading a revolution against the regime. We need all the help we can get.
John: I'm ready to do whatever it takes. What can I do?
Samantha: We need to gather more people to join our cause. We need to spread the word and convince people to join us.
John: I can do that. I'll start talking to people and see if I can get them to join us.
Samantha: That would be great. We need all the help we can get.
John: We can do this! We can take back our freedom!
Samantha: Yes, we can! We

KeyboardInterrupt: 

In [15]:
for dialogue in dialoguesSoFar:
        print(f"{dialogue['name']}: {dialogue['text']}")

John: Hi, I'm John. It's nice to meet you.
Jane: Hi, I'm Jane. It's nice to meet you too.
John: So, what brings you here?
Jane: I'm here to fight for justice and to overthrow the oppressive government.
John: Me too! We should join forces and work together to make a difference.
Jane: That sounds like a great idea. Let's do it!
John: We need to come up with a plan that will help us overthrow the oppressive government.
Jane: Yes, we need to be strategic and think of every possible outcome.
John: We should start by gathering as much information as we can about the government and their plans.
Jane: Good idea. We should also start recruiting people to join our cause.
John: Yes, we need to build a strong team if we are going to succeed.
Jane: We should also start stockpiling weapons and supplies in case we need them.
John: Right, we need to be prepared for anything.
Jane: We can do this. We just need to stay focused and keep our eyes on the prize.
John: We need to be careful. The government i